In [45]:
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from api_util import read_token, BASE_URL, get_headers
import requests

def _make_get_request(endpoint, params=None, quiet=None):
    """共用的 HTTP GET 請求處理函數"""
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=get_headers(read_token()), params=params)
        response.raise_for_status()
        result = response.json()
        if not quiet: 
            print(result)
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None
    
def _make_post_request(endpoint, data, quiet=None):
	try:
		response = requests.post(f"{BASE_URL}/{endpoint}", headers=get_headers(read_token()), json=data)
		response.raise_for_status()
		result = response.json()
		if not quiet: 
			print(result)
		return result
	except requests.exceptions.RequestException as err:
		print(f"Request failed: {err}")
		return None
            
def get_all_momentum():
    res = _make_get_request('market/momentum')
    return res.get('data')

def get_momentum_by_range(days):
    res = _make_get_request(f'market/momentum/range/{days}')
    return res.get('data')
    
def get_market_weights():
    res = _make_get_request('market/weights')
    return res.get('data')

def get_stock_symbols():
    res = _make_get_request('market/stock/symbols')
    return res.get('data')

def get_market_breadth():
    res = _make_get_request('market/stock/breadth')
    return res.get('data')

def get_stock_winners():
    res = _make_get_request('market/stock/winners')
    return res.get('data')

def get_stock_losers():
    res = _make_get_request('market/stock/losers')
    return res.get('data')

def get_statement_by_symbol(symbol):
    return _make_get_request(f"statements/{symbol}")

def get_market_data_by_symbol(symbol, page=1, size=10):
    res = _make_get_request(f'market/{symbol}', params={'page': page, 'size': size})
    return res.get('data')


def draw_line_chart(x_data, y_data, label, title, xlabel, ylabel, interval=10, figsize=(10, 6), color='#efb441'):
    plt.figure(figsize=figsize)
    plt.plot(x_data, y_data, marker='', linestyle='-', color=color, label=label)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
    plt.gcf().autofmt_xdate()
    plt.grid(True)

    plt.legend()

    plt.tight_layout()
    plt.show()
    
def plot_pe_forward_vs_time(data):
    timestamps = []
    pe_forward_values = []
    
    for entry in data:
        created_at = entry.get('createdAt', '')
        pe_forward = entry.get('PE_Forward', None)
        
        if created_at and pe_forward is not None:
            timestamp = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
            timestamps.append(timestamp)
            
            pe_forward_values.append(float(pe_forward))
    
    draw_line_chart(timestamps, pe_forward_values, label='PE Forward', title='PE Forward vs Time', xlabel='Date', ylabel='PE Forward', figsize=(7, 5))
    



In [ ]:
symbol = 'BTCUSD'
res = get_market_data_by_symbol(symbol, page=2, size=2)
res

In [ ]:
res = get_market_weights()
res

In [ ]:
breadth = get_market_breadth()
breadth

In [ ]:
my_data = get_all_momentum()
for i in range(7):
	print(my_data[i])

In [ ]:
my_data = get_momentum_by_range(7)
for i in range(7):
	print(my_data[i])

In [ ]:
import pandas as pd

df = pd.DataFrame(my_data)

df['createdAt'] = pd.to_datetime(df['createdAt'])
draw_line_chart(df['createdAt'], df['volume'], label='Momentum', title='Time Series Momentum', xlabel='Time', ylabel='Volume', interval=1)

In [ ]:
winners = get_stock_winners()
for stock in winners:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

In [ ]:
losers = get_stock_losers()
for stock in losers:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

In [ ]:
m7 = ['nvda', 'goog', 'meta', 'amzn', 'tsla', 'msft', 'crwd', 'pltr', 'coin']

for item in m7:
	res = get_statement_by_symbol(item)
	data = res.get('data')
	plot_pe_forward_vs_time(data[-500:])